ChatGPT is a word-level language model that has taken the world by storm. In this notebook, we will focus on what is under the hood of ChatGPT. What is the neural network under the hood that models the sequence of words?

ChatGPT is an attention-based architecture -- GPT is short for Generatively Pretrained Transformer. The paper "[Attention Is All You Need](https://arxiv.org/abs/1706.03762)" was a landmark paper in AI that proposed the Transformer architecture. The paper reads like a random machine translation paper. The authors probably didn't fully anticipate the impact that the architecture would have on the whole field of ML. The architecture they proposed in the context of machine translation ended up taking over the rest of AI since it has been published. The architecture was copy-pasted with minor changes into a huge number of applications, including ChatGPT. This neural net architecture handles the heavy load in ChatGPT.

We're gonna build *something like* ChatGPT. Of course, we're not going to be able to reproduce ChatGPT. It is a very serious production-grade system trained on a good chunk of the internet, and there is a lot of pre-training and fine-tuning to it. Instead, we're going to build and train a Transformer-based character-level language model, which will be very educational wrt. how the actual ChatGPT works. We also won't train on a good chunk of the internet -- we are going to use a smaller dataset. In particular, our choice will be Andrej's favorite toy dataset, the "Tiny Shakespeare" dataset.    

Given some context of characters in the past (the prompt), the Transformer NN will look at the context and predict the next token (character) of the sequence. We're going to maximize the likelihood of the dataset under the model parameters. In the actual ChatGPT, the tokens do not correspond to characters, they correspond to word "chunks", i.e., sub-word pieces.

We will follow the [nanoGPT] repository of Andrej, which is a repo for training Transformers on any given text. There are many ways to train Transformers, this is probably the most simple one. It only contains 2 files of 300 lines each. One of them defines the GPT model (the Transformer) and the other one trains it on a given text dataset. If we train the model on OpenWebText (a fairly large dataset of webpages), then we can reproduce the performance of GPT-2 124M (which is an early version of openAI's GPT from 2017), which proves that the codebase is correctly arranged. We can also load the GPT-2 124M weights that openAI released.

Now we're going to basically write the repository from scratch. First, we start with the ``train.py`` file. We will define a Transformer piece by piece, and we're going to train it on the TinyShakespeare dataset. We'll see how we can then generate infinite Shakespeare.

In [571]:
from karpathy_nn.makemore.data.load_data import load_shakespeare

import torch
from torch import Tensor

from torch import nn
from torch.nn import functional as F
from torch.nn import Module

from typing import Optional
from torch.optim import AdamW


In [572]:
text = load_shakespeare()
print(f"Length of dataset in characters: {len(text):,}")


Length of dataset in characters: 1,115,394


Let's look at the first 1000 characters of the dataset.

In [573]:
print(text[:1000])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Let's check the unique characters that occur in the entire text. These are all possible characters that the model can see/emit.

In [574]:
characters = sorted(list(set(text)))
num_tokens = len(characters)
print("".join(characters))
print(num_tokens)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Now we will tokenize the input text. We will convert the raw text (a single string) into a sequence of integers according to some vocabulary of possible elements. As we're building a character-level language model, we will be converting the individual characters into integers.

In [575]:
# Create a mapping from characters to integers
string_to_integer = {character: integer for integer, character in enumerate(characters)}
integer_to_string = {integer: character for integer, character in enumerate(characters)}


def encode(string: str) -> list[int]:
    return [string_to_integer[character] for character in string]


def decode(int_list: list[int]) -> str:
    return "".join([integer_to_string[integer] for integer in int_list])


print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


This is a very simple text encoding/tokenizer. People have come up with many tokenizer methods. E.g., Google uses [SentencePiece](https://github.com/google/sentencepiece), which also encodes text into integers, but in a different schema and using a different vocabulary. It is a **sub-word tokenizer**, i.e., we're not encoding entire words, but we're also not encoding individual characters. This is usually what's used in practice. OpenAI uses [tiktoken](https://github.com/openai/tiktoken), which is a **byte pair encoding tokenizer**. This is what GPT uses. Instead of having just 65 tokens, they have 50257 tokens. When we encode the string "hii there", we only get a list of three integers, that are between 0 and 50256. There is a trade-off between the code book size and the tokenized sequence lengths. In summary, people in practice use sub-word tokenizers, but we will keep our tokenizer very simple and we will only build a character-level language model. (Small code book, simple encode and decode, long encoded sequences.)

Now we're ready to tokenize the entire TinyShakespeare dataset.

In [576]:
# Let's encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(
    "The 1000 characters we looked at earlier will look like this to GPT-2", data[:1000]
)


torch.Size([1115394]) torch.int64
The 1000 characters we looked at earlier will look like this to GPT-2 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59, 

Let's now split our dataset into training and validation sets.


In [577]:
n = int(0.9 * len(data))  # First 90% will be trainined data, rest will be val
training_data = data[:n]
val_data = data[n:]


By looking at the validation performance, we can get a good idea about overfitting. In particular, we don't want the network to memorize the training data -- we want it to produce training-data-like text, and we want the validation text to have a high probability under our model. Of course, we can also overfit to the validation set itself.

When we train the Transformer, we sample little chunks from the training set and train the network to predict the GT next token given the context from the chunks. The maximum length of the chunks is typically called the block size or the context length.

In [578]:
block_size = 8
training_data[: block_size + 1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

This chunk actually has multiple examples packed into it, as all characters follow each other. We are going to simultaneously train the Transformer to make predictions at every one of these positions. In a chunk of 9 characters, we have 8 individual examples:
- Given {18}, 47 likely comes next.
- Given {18, 47}, 56 likely comes next.
- Given {18, 47, 56}, 57 likely comes next.
- ...
- Given {18, 47, 56, 57, 58, 1, 15, 47}, 58 likely comes next.

In [579]:
x = training_data[:block_size]
y = training_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"When input is {context.tolist()}, the target is {target}.")


When input is [18], the target is 47.
When input is [18, 47], the target is 56.
When input is [18, 47, 56], the target is 57.
When input is [18, 47, 56, 57], the target is 58.
When input is [18, 47, 56, 57, 58], the target is 1.
When input is [18, 47, 56, 57, 58, 1], the target is 15.
When input is [18, 47, 56, 57, 58, 1, 15], the target is 47.
When input is [18, 47, 56, 57, 58, 1, 15, 47], the target is 58.


These are the 8 examples hidden in the chunk of 9 characters that were sampled from the training set. We train on all of these examples, with context length between 1 and 8. We don't only train on all of them for computational reasons (because we have the sequence already), it's also done to make the transformer network be used to seeing contexts from size one to size ``block_size``. That's going to be very useful during inference, because while we're sampling, we can start the sampling generation with as little as one character of context. Then the Transformer will know how to predict the next character even given such a small context. After ``block_size``, we'll have to start truncating, because the Transformer will never receive more than ``block_size`` inputs at once.

So far we've only looked at the time dimension. We also have to consider the batch dimension. When we feed the examples to the Transformer, we are actually going to feed *mini-batches* of multiple chunks of text that are all stacked up into a single tensor. This is all just done for efficiency, because GPUs are very good at parallel processing of data. The chunks are processed completely independently -- they don't influence each other, unlike in BatchNorm.

In [580]:
torch.manual_seed(1337)
# How many independent sequences will we process in parallel?
batch_size = 4

# What is the maximum context length for predictions?
block_size = 8


def get_batch(split: str) -> tuple[Tensor, Tensor]:
    # Generate a small batch batch of data of inputs x and targets y
    data = training_data if split == "train" else val_data

    # Start index of chunks
    idx = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i : i + block_size] for i in idx])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in idx])

    return x, y


xb, yb = get_batch("train")
print("Inputs:")
print("\t", tuple(xb.shape))
print("\t", xb)
print("Targets:")
print("\t", tuple(yb.shape))
print("\t", yb)

print("-" * 8)

for batch in range(batch_size):  # Batch dimension
    for t in range(block_size):  # Time dimension
        context = xb[batch, : t + 1]
        target = yb[batch, t]
        print(f"When input is {context.tolist()}, the target is {target}.")


Inputs:
	 (4, 8)
	 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
	 (4, 8)
	 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------
When input is [24], the target is 43.
When input is [24, 43], the target is 58.
When input is [24, 43, 58], the target is 5.
When input is [24, 43, 58, 5], the target is 57.
When input is [24, 43, 58, 5, 57], the target is 1.
When input is [24, 43, 58, 5, 57, 1], the target is 46.
When input is [24, 43, 58, 5, 57, 1, 46], the target is 43.
When input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39.
When input is [44], the target is 53.
When input is [44, 53], the target is 56.
When input is [44, 53, 56], the target is 1.
When input is [44, 53, 56, 1], the target is 58.
When input is [44, 53, 56, 1, 58],

The Transformer is going to simultaneously process all of these examples, and then it will predict (hopefully the correct) characters for *every* position in a single sample.

In [581]:
print(xb)  # Our input to the transformer


tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [582]:
torch.manual_seed(1337)


class BigramLanguageModel(Module):
    def __init__(self, num_tokens: int) -> None:
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(num_tokens, num_tokens)

    def forward(self, idx: Tensor, targets: Optional[Tensor] = None) -> Tensor:
        # idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx)  # (B, T, C)
        # e.g., (4, 8, 65)

        if targets is None:
            loss = None
        else:
            # PyTorch wants a (B, C) or a (B, C, T) input!
            B, T, C = logits.shape
            logits = logits.reshape(B * T, C)
            targets = targets.reshape(B * T)

            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Get the predictions
            logits, loss = self(idx)  # (B, T, C), ()
            # Focus only on the last time step
            # The history is not used at all, only T - 1 -> T
            # so this looks silly, but we want to keep the
            # structure of the dataset, as the history will
            # be used later on.
            logits = logits[:, -1, :]  # Becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append sampled inde to the running sequence
            idx = torch.cat([idx, idx_next], dim=1)  # (B, T + k)

        return idx


model = BigramLanguageModel(num_tokens=num_tokens)
logits, loss = model(xb, yb)
print(yb.shape, logits.shape, loss)
idx = torch.zeros((1, 1), dtype=torch.long)  # Batch 1, time 1, zero is a newline
idx = model.generate(idx, max_new_tokens=100)[0].tolist()  # (101,)
print(decode(idx))


torch.Size([4, 8]) torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


We are expecting the loss to be $-\log(1/65) \approx 4.174$. As we have a notable discrepancy, this shows that we are guessing wrong (we have a little bit of entropy) initially. Our initialization is not perfect, but it might be just noise coming from the small number of samples. The generated string is garbage, but that's what we expect from an untrained model. Let's train the model.

Note: In makemore we only used SGD. AdamW is a much more advanced and popular optimizer.
A typical good learning rate for AdamW is 1e-3, but for very small nets (like here),
we can get away with higher learning rates.

In [583]:
# Create a PyTorch optimizer
optimizer = AdamW(model.parameters(), lr=1e-3)  # TODO


In [584]:
batch_size = 32
for steps in range(10000):
    # Sample a batch of data
    xb, yb = get_batch("train")

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=False)
    loss.backward()
    optimizer.step()

print(loss.item())


2.382369041442871


In [585]:
idx = torch.zeros((1, 1), dtype=torch.long)
idx = model.generate(idx, max_new_tokens=500)[0].tolist()
print(decode(idx))



lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


Well, this is not Shakespeare, but also not complete gibberish. The model is making some progress.

Note that the tokens are not "talking" to each other: we're only making our predictions only based on the very last token. Now we want to make them "talk" to each other. This is how we arrive at Transformers.

We're almost ready to start writing our very first self-attention block for processing the tokens. Before that, we will look at a **mathematical trick used in self-attention**. This is at the heart of an efficient implementation of self-attention. Let's see a toy example first.

In [586]:
# Consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2  # Batch, time, channels
x = torch.randn(B, T, C)
x.shape


torch.Size([4, 8, 2])

We would like the 8 tokens in a single batch to "talk to each other". We would like to couple them. In particular, we want to couple them in a very specific way. The token at the 5th location should not communicate with subsequent tokens, because those are future tokens in the sequence. The token at the 5th location should only communicate with previous tokens. **Information should only flow from previous context to the current time step.** We can't get any information from the future, because we want to *predict* the future.

What is the easiest way for tokens to communicate? If I am the 5th token and would like to communicate with my past, the simplest way to do that is to just do an average of all preceding elements. In particular, we want to average the channels for the 1st, 2nd, 3rd, 4th, and 5th time steps. That becomes a feature vector that summarizes me and my history.

Of course, doing an average is an extremely weak form of interaction -- this communication is very lossy. We lose all information about the spatial arrangement of all tokens. But that's OK for now. We'll see how we can bring that information back later.

What we want to do is as follows.
- For every single batch element independently, ...
- ..., for every t-th token in the sequence ...,
- ..., we want to calculate the average of all the vectors at the current or previous locations.

In [587]:
# We want x_bow[batch, t] = mean_{i <= t} x[batch, i]
# BoW = Bag of Words -- term that people use when you just average up things
x_bow = torch.zeros((B, T, C))
for batch in range(B):
    for t in range(T):
        x_prev = x[batch, : t + 1]  # (t, C)
        x_bow[batch, t] = x_prev.mean(dim=0)


In [588]:
x[0]


tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [589]:
x_bow[0]


tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

We can see that we're basically performing a ``.cumsum``, but with averaging instead of summing. The trick is that we can be very efficient in computing this using matrix multiplication.

In [590]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a =")
print(a)
print("-" * 8)
print("b =")
print(b)
print("-" * 8)
print("c =")
print(c)


a =
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--------
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--------
c =
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


We get a column-wise sum of ``b``. ``torch.tril`` returns the lower-triangular portion of an array.

In [591]:
torch.tril(torch.ones(3, 3))


tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [592]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a =")
print(a)
print("-" * 8)
print("b =")
print(b)
print("-" * 8)
print("c =")
print(c)


a =
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--------
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--------
c =
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


We get a column-wise ``.cumsum``! If we normalize the rows of ``a``, we get a ``.cumavg``.

In [593]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / a.sum(dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a =")
print(a)
print("-" * 8)
print("b =")
print(b)
print("-" * 8)
print("c =")
print(c)


a =
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--------
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--------
c =
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


This is the column-wise ``.cummean`` of ``a``. Let's vectorize the calculation of ``x_bow``.

In [594]:
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(dim=1, keepdim=True)
x_bow2 = weights @ x  # (T, T) @ (B, T, C) --> (B, T, T) @ (B, T, C) = (B, T, C)
# This is a batched matmul, which implements a weighted cumsum!!!
torch.allclose(x_bow, x_bow2)


True

We'll see one more way to calculate ``x_bow``. This is a bit more interesting than the previous approaches. The reason why we'll end up using it in self-attention: the weights begin with 0. We can think of this as an interaction strength or an affinity. It's telling us how much of each token from the past do we want to aggregate and average up.

In [595]:
# Version 3: use softmax
tril = torch.tril(torch.ones(T, T))
weights = torch.zeros((T, T))  # The affinities are just set by us to be zeros.
# Quick preview: these affinities between the tokens are not going to be just constant
# zeros, they are going to be data-dependent. The tokens will start looking at each
# other, and some tokens will find other tokens more or less interesting. Depending on
# what their values are, they are going to find each other interesting to different
# extents.

# Row i of weights is telling us how much each other token (columns) count in the
# aggregation of embedding values.
# The subsequent lines will stay the same in self-attention.

# This line is saying: tokens from the future cannot communicate.
# By setting them to -inf we're saying that we won't aggregate anything from
# those tokens.
weights = weights.masked_fill(tril == 0, -float("inf"))
weights = F.softmax(weights, dim=-1)

# Weighted (now uniform) agrgregation of past elements
# through (batched) matrix multiplication
x_bow3 = weights @ x
# We aggregate the token embeddings depending on how interesting they find each other.
torch.allclose(x_bow, x_bow3)


True

We're going to use this trick to implement the self-attention block. We start with the crux of self-attention. This is probably the most important part of this notebook to understand. We'll implement a small self-attention for a single individual "head", as they're called.

In [596]:
# Version 4: self-attention
torch.manual_seed(1337)

# 4 x 8 arrangement of tokens
# Embedding of each token is 32D
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T, T))

# Previously, we initialized the affinities between all tokens/nodes to be zero
# (uniform).
# This corresponds to a simple masked average.
# Now, we don't want this to be uniform. Different tokens will find different other
# tokens more or less interesting, and we want that to be data_dependent.
# E.g., if I'm a vowel, then maybe I'm looking for consonants in my past. Maybe I want to
# know what those consonants are and I want that information to flow to me. Thus, I want
# to gather information from the past, but I want to do it in a data dependent way.
# This is the problem that self-attention solves.

# Every single token at each position will emit two vectors. It will emit a query, and a
# key.
# The query vector encodes "What am I looking for?".
# The key vector encodes "What do I contain?".
# We get affinities between the tokens in the sequence by doing a dot product between
# the keys and the queries. My query . all the other keys = weights. If the key and the
# query are aligned, they will interact by a large amount. Then I will get to learn more
# about that specific token compared to the other tokens in the sequence. Let's implement
# this. Actually, we will implement a single head of self-attention.

# Let's see a single head perform self-attention

# Hyperparameter of the head -- influences the dimensionality of the vectors
# we are going to take the dot products of.
head_size = 16

# Just matmul with some weights
key = nn.Linear(in_features=C, out_features=head_size, bias=False)  # (C, head_size)
query = nn.Linear(in_features=C, out_features=head_size, bias=False)  # (C, head_size)

# When we push through our mini-batch through key and query, we're processing all tokens
# in all positions of the (B, T) arrangement *independently* to produce a key and a
# query. No communication is happening. This is embarrassingly parallelizable.
k = key(x)  # (B, T, head_size)
q = query(x)  # (B, T, head_size)

# The communication comes now. All the queries will dot product with all the keys.
# weights gives us the (T, T) non-symmetric affinities for each batch element.
weights = q @ k.transpose(-2, -1)  # (B, T, head_size) @ (B, head_size @ T) = (B, T, T)
print(torch.allclose(weights, weights.transpose(-2, -1)))  # Not symmetric!
# Take a fixed batch b, t' from q, take t'' from k, measure their dot product.
# Place it at weights[b, t', t''].
# Rows of weights correspond to the queries, columns correspond to the keys.
# We keep the batch dimension untouched -- the tokens across samples can't communicate.

# Up until now we have the raw affinities between each node.
# Now we will make sure that we don't look into the future and aggregate information
# from subsequent nodes. We're not allowed to communicate with the future.

weights = weights.masked_fill(tril == 0, -float("inf"))
weights = F.softmax(weights, dim=-1)

# There is one more part to a single self-attention head.
# When we do the aggregation, we don't actually aggregate the tokens exactly. We
# produce one more value, called the "value". :)

# Can be of different dimensionality than queries and keys
value = nn.Linear(C, head_size, bias=False)
v = value(x)  # The elements that we aggregate instead of the raw x values.
# We can think of x as private information to each token. Each token's information is
# kept in vector x. "**For the purposes of this single head**, here's what I'm interested
# in, here's what is important about me for this head, and if you find me interesting,
# this is what I'll communicate to you." This is stored in v. It's just the thing that
# gets aggregated for the purposes of this single head between the different nodes.

# out = weights @ x
out = weights @ v  # (B, T, T) @ (B, T, head_size) = (B, T, head_size)

# The weighted aggregation now is a function in a data-dependent manner of the keys and
# queries of these nodes. It answers how much informations a query node should aggregate
# from any of the tokens in the past.

# This is basically the self-attention mechanism.

out.shape


False


torch.Size([4, 8, 16])

In [597]:
weights[0]


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Previously, weights was just a constant -- we applied a uniform aggregation to all tokens
in the same batch element. Now, every single node in a single batch element has a
different weight wrt. a fixed query node, because everything is data dependent.

Example: take the 8th token. It knows what content it has and what position it's in.
Based on that, the token creates a query: "Hey, I'm looking for this kind of stuff! I'm
a vowel, I'm in the 8th position, I'm looking for any consonants at position up to four.
". (Just an example.) All nodes emit keys. Maybe a node's key will say "I'm a consonant!
I'm in a position up to four!". That key, query dot product will have a high value, a
high *affinity*. When two nodes have a high affinity, then through the softmax the query
node will aggregate a lot of the corresponding key node's information. This is the case
e.g. for the query of the 8th node and the key of the 4th node with 22.97% matching
probability.

A few notes about attention:
- Attention is a **communication mechanism**. It can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights. The nodes of the graph are laid out linearly, with the i-th node having arrows pointing into it from all previous nodes and itself. That's the structure that our graph has in autoregressive scenarios such as language modeling. But in principle, attention can be applied to any arbitrary directed graph and is just a communication mechanism between the nodes.
- There is no notion of space. Attention simply acts over a set of vectors. The nodes have no clue where they are positioned in the space. This is why we need to positionally encode tokens and give them some information that is anchored to a specific position such that they know where they are. This is different from convolution, because if we run the convolution operation over some input, there is a very specific layout of the information in space, and the convolutional filters are acting in the space. Attention just makes a set of vectors communicate, and if we want them to have a notion of space, we need to add it explicitly. This is what we've done when we calculated the positional encoding and added that information to the vectors.
- Each example across the batch dimension is, of course, processed completely independently and never "talk" to each other. We have ``batch_size`` separate pools of ``T`` nodes in our graph.
- In the case of language modeling, we have a specific structure of a directed graph where the future tokens' informations are not aggregated into the current token's information pool. This is not a general constraint. In many cases we want to have all of the nodes talk to each other fully. E.g., if we're doing sentiment analysis with a Transformer, we might have a number of tokens and we might want all of them to talk to each other fully, because later we're predicting the sentiment of the sentence, having seen the entire sentence. In an "encoder" (self-)attention block, we just have to delete the single line that does masking with `tril`, allowing all tokens to communicate with each other. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling (where it's decoding language -- nodes in the future never talk to the current node, as they don't exist yet). Here we have a triangular structure. But both are allowed, and attention doesn't care. Attention supports arbitrary connectivity between nodes.
- "self-attention" just means that the keys and values are produced from the same source x as queries. In principle, attention is much more general than that. In "cross-attention" (e.g., in encoder-decoder transformers), the queries still get produced from x, but the keys and values come from some other, external source (e.g., an encoder module/block that encodes some context we'd like to condition on). So the keys and values will come from a whole separate source (nodes on the side), and here we're just producing queries and we're reading off information from the side. Cros-attention is thus used when there is a separate source of nodes we'd like to pull information from into our nodes. The attention above is thus self-attention.
- "Scaled" attention additionally divides ``weight`` by $1 / \sqrt{\text{head\_size}}$. This makes it so when input Q, K are unit variance, ``weight`` will be unit variance too and Softmax will stay diffuse and not saturate too much. If we don't do it, then we will be aggregating information from 1-2 other nodes. This is not we want, especially at initialization. The scaling is done to control the variance at initialization. It is applied even during training to smoothen the softmax predictions. Illustration below.

In [598]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)

# Multiplying by this is harmful: * 1 / T**0.5
v = torch.randn(B, T, head_size)
weights = q @ k.transpose(-2, -1)


In [599]:
k.var()


tensor(1.0449)

In [600]:
q.var()


tensor(1.0700)

The product of individual elements in the matmul $X \sim \mathcal{N}(0, 1)$ and $Y \sim \mathcal{N}(0, 1)$ will not be Gaussian distributed, but whatever their distribution is, their variance is
$$
\begin{align*}
\text{Var}(XY) &= \mathbb{E}[X^2Y^2] - \mathbb{E}[XY]^2\\
&= \int_{\mathcal{X} \times \mathcal{Y}} p(x, y) x^2y^2 d(x, y) - \left(\int_{\mathcal{X} \times \mathcal{Y}} p(x, y) xy d(x, y)\right)^2\\
&= \int_{\mathcal{X}} p(x) x^2 dx \int_{\mathcal{Y}} p(y) y^2 dy - \left(\int_{\mathcal{X}} p(x)xdx \int_{\mathcal{Y}} p(y)ydy\right)^2\\
&= \mathbb{E}[X^2]\mathbb{E}[Y^2] - \mathbb{E}[X]^2\mathbb{E}[Y]^2\\
&= \left(\mathbb{E}[X^2] - \mathbb{E}[X]^2\right)\left(\mathbb{E}[Y^2] - \mathbb{E}[Y]^2\right) + \mathbb{E}[X^2]\mathbb{E}[Y]^2 + \mathbb{E}[Y^2]\mathbb{E}[X]^2 - 2\mathbb{E}[X]^2\mathbb{E}[Y]^2\\
&= \text{Var}(X)\text{Var}(Y)\\
&= 1,
\end{align*}
$$
as the variables are *independent*. For the same reason, their expectation is
$$
\begin{align*}
\mathbb{E}[XY] &= \int_{\mathcal{X} \times \mathcal{Y}} p(x, y) xy d(x, y)\\
&= \int_{\mathcal{X}}p(x)xdx \int_{\mathcal{Y}}p(y)ydy\\
&= \mathbb{E}[X]\mathbb{E}[Y]\\
&= 0.
\end{align*}
$$

 We also know that the sum of any two independent random variables $Z$ and $W$ has variance
$$
\begin{align*}
\text{Var}(Z + W) &= \int_{\mathcal{Z} \times \mathcal{W}} p(z, w) \left(z + w - \mathbb{E}[Z + W]^2\right)^2dzdw\\
&= \int_{\mathcal{Z}}\int_{\mathcal{W}}p(z)p(w) \left((z - \mathbb{E}[Z]) + (w - \mathbb{E}[W])\right)^2dzdw\\
&= \int_{\mathcal{Z}}\int_{\mathcal{W}}p(z)p(w)\left((z - \mathbb{E}[Z])^2 + 2(z - \mathbb{E}[Z])(w - \mathbb{E}[W]) + (w - \mathbb{E}[W])^2\right)dzdw\\
&= \int_{\mathcal{Z}} p(z)(z - \mathbb{E}[Z])^2dz + \int_{\mathcal{W}}p(w)(w - \mathbb{E}[W])^2 dw + 2\int_{\mathcal{Z}}p(z)(z - \mathbb{E}[Z])dz \int_{\mathcal{W}}p(w)(w - \mathbb{E}[W])dw\\
&= \text{Var}(X) + \text{Var}(Y).
\end{align*}
$$

If we now consider the dot product between two independent random vectors $X \sim \mathcal{N}(0, I_n), Y \sim \mathcal{N}(0, I_n)$, we get
$$\text{Var}(X^\top Y) = \text{Var}\left(\sum_{i = 1}^n X_iY_i\right) = \sum_{i = 1}^n \text{Var}\left(X_iY_i\right) = n$$
and
$$\mathbb{E}[X^\top Y] = 0.$$
If we consider a matrix multiplication, this dot product will give a single element of the result.

In [601]:
# The population variance is exactly 16.
weights.var()  # On the order of head_size = 16 (= n)


tensor(17.4690)

In [602]:
weights = F.softmax(weights, dim=1)
print(
    weights.min(),
    weights.max(),
    weights.mean(),
    torch.logical_or(weights > 0.95, weights < 0.05).float().mean(),
)

out = weights @ v
out.var()


tensor(3.0237e-11) tensor(0.9997) tensor(0.1250) tensor(0.7422)


tensor(3.0374)

To not make the elements of the weights tensor explode and have confidently wrong (we still have randomness) predictions in the softmax, we have to devide the elements of weights by $\sqrt{\text{head\_size}}$. By doing so, the weights tensor will have unit variance again, making it well-behaved under the initialization.

In [603]:
weights = q @ k.transpose(-2, -1) * head_size**-0.5

print(weights.var())  # Unit variance is preserved
weights = F.softmax(weights, dim=1)
print(
    weights.min(),
    weights.max(),
    weights.mean(),
    torch.logical_or(weights > 0.95, weights < 0.05).float().mean(),
)

out = weights @ v
out.var()


tensor(1.0918)
tensor(0.0014) tensor(0.8043) tensor(0.1250) tensor(0.2812)


tensor(1.1150)

The entire network is now implemented in ``gpt.py``.

We did a pretty good job of implementing the Transformer architecture, but the Figure in the paper doesn't exactly match up to what we've been doing. What's going on with all the additional parts?

What we implemented here is a decoder only Transformer. There's no encoder component, and there is also no cross-attention block. Our block only has a self-attention and a feed-forward. It is missing the third in-between piece. The reason why we only have a decoder only is that we are just generating text and it's unconditioned on anything. We're just blabbering on, according to a given dataset. What makes our network a decoder is that we're using the triangular mask, so it has this autoregressive property where we can just go and sample from it. It can be used for language modeling. We just have a dataset and we want to imitate it, which is why we're using a decoder only transformer. This is exactly as done in GPT.

In machine learning, autoregressive refers to a class of models that make predictions based on previous values of the same variable. Autoregressive models are commonly used in time series analysis, where the goal is to make predictions based on patterns observed in past data. For example, a time series model that predicts stock prices might use an autoregressive approach to capture trends and seasonality in the data.

The reason that the original paper had an encoder-decoder architecture is because it is a machine translation paper. It is concerned with a different setting. In particular, it expects some tokens that encode, e.g., a French sentence, and then it is expected to decode the translation in English. Example:

<------------- ENCODE -------------> | <------------------ DECODE ----------------->

Les réseaux de neurones sont géniaux! | \<START\>Neural networks are awesome!\<END\>

\<START\> and \<END\> are special tokens. The network is expected to read in the French sentence and condition on it, and then we start off the generation with the special \<START\> token which we always place in the beginning. Then the network is expected to output the English translation of the French sentence and the special \<END\> token to finish the generation. Here, the decoder will work exactly as we implemented it. It is first conditioned on the \<START\> token, then the \<START\> and the newly generated token (not necessarily "Neural"), and so on. But unlike how we implemented transformers, they condition the generation of tokens on some additional information. In their case, this additional information is the French sentence that they should be translating. The encoder reads the French sentence (after tokenizing it) and puts a Transformer on top, but *without* the triangular mask in the self-attention layers. All tokens are allowed to talk to each other as much as they want. They are just encoding the content of the French sentence. Once they've encoded it, they come out on the top in shape (B, T, C). In our decode (which does the language modeling), there is an additional connection to the outputs of the encoder. This connection is the cross-attention between the encoder output representation and the decoder representation. The queries are still generated from x, but the keys and values are coming from the "side", i.e., they are generated from the output nodes of the encoder. These keys and values feed into every single block of the decoder. Thus, the decoder is conditioned not just on the past of the current sentence, but also on the fully encoded French prompt.

In summary, the original architecture was used for machine translation (encoder-decoder model), and that's why we have those two Transformers and an additional block in the decoder.

### A walkthrough of nanoGPT

nanoGPT is basically two files of interest, ``train.py`` and ``model.py``. ``train.py`` is the boilerplate code for training the network. It's very similar to our training loop, but it's a lot more complicated:
- We are saving and loading checkpoints and pretrained weights
- We are decaying the learning rate
- We are compiling the model
- We are using distributed training across multiple nodes of GPUs
- etc.

However, the ``model.py`` should look very-very similar to what we've done here. In fact,
the models are almost identical. The ``CausalSelfAttention`` block implements masked self-attention.

What's different is that in our code, we've separated the multi-head attention into just a single individual head, and we explicitly concatenate these in the multi-head attention module. In nanoGPT, all of that is implemented in a batched manner inside a single causal self-attention block. Thus, we don't just have a B, T, and C dimension, we end up with a fourth dimension that is the heads. The two approaches are mathematically equivalent, nanoGPT is just much more efficient because all the heads are now treated as a batch dimension as well. The MLP uses the gelu non-linearity instead of ReLU. This is because openAI also used it and Andrej wanted to be able to load their checkpoints. The blocks of the Transformer are identical (communicate and compute phase), and GPT is also identical. We have the positional encodings, token encodings, the Blocks, the LayerNorms, etc. There are some more things, e.g., Andrej separates out the weights that should be weight decayed and ones that shouldn't. So a few details are different, but we should understand most things.

### ChatGPT

What would it look like if we wanted to train ChatGPT ourselves? How does it relate to what we've learned today? To train ChatGPT, there are roughly two stages, pre-training and fine-tuning. In the pre-training stage, we are training on a large chunk of the internet and just trying to get a great decoder-only Transformer to babble text. This part is very-very similar to what we've done ourselves, except we've done a baby pre-training step. Our upscaled model has about 10M parameters, so this Transformer is quite tiny compared to today's standards. Our dataset was roughly 1M characters/tokens. OpenAI uses a different vocabulary -- they're not on the character level, they use sub-word chunks and have a vocabulary of 50K elements. Their sequences therefore are a bit more condensed. The Shakespeare dataset would be around 2-300K tokens in the OpenAI vocabulary, so we trained a 1M parameter model on roughly 300K tokens. If we go to the GPT-3 paper ([Language Models are Few-Shot Leaners](https://arxiv.org/abs/2005.14165)) and look at Table 2.1, we see that the biggest Transformer had 175B parameters, they used 96 layers, ``dim_embedding = 12288``, ``num_heads = 96``, (so ``head_size = dim_embedding / num_heads = 128``). Their batch size was 3.2M (!!!), compared to our 64. Their learning rate was 0.6e-4. All models were trained for a total of 300B tokens (!!!). This number is not even large for today's standards, it would be 1 trillion and above.

So they're training a significantly larger model on a good chunk of the internet. That's the pretraining stage. But the architecture is nearly identical to what we've implemented, the difference is that they are using much larger hyperparameters. Obviously, it's a massive infrastructure challenge to train this. We're talking 1000s of GPUs having to talk to each other to train models of this size.

After we complete the pretraining stage, we don't get something that responds to our questions with answers. We only get a *document completer*. It babbles, but it doesn't babble Shakespeare, it babbles the internet. It will create arbitrary news articles, it'll try to complete documents, etc. This is what it was trained to do: to complete the sequence. If we gave it a question, it would potentially just follow up with more questions. It would do whatever some closed document would do in the training data. We're getting undefined behavior: it could actually also answer our question if it saw that kind of data. It can also ignore our question, it can start a totally different news article, etc. It's totally *underlined* as we say.

The second fine-tuning stage is to actually align ChatGPT to be an assistant. The [ChatGPT blog post](https://openai.com/blog/chatgpt/) from OpenAI talks a little bit about how this stage is achieved. There are roughly three steps to this stage.

(1) They start to collect training data that looks specifically like what an assistant would do. They are documents that have the format where the question is on top and an answer is below. They have a large number of these, but of course, not on the order of the size of the internet. This is on the order of maybe thousands of examples. They then fine-tune the model to only focus on documents that look like that. So we're starting to slowly align it. It's going to expect a question at the top and it's going to complete the prompt with the answer. These very-very large models are very sample efficient during their fine-tuning, so this actually works.

(2) They let the model respond to queries, and then different raters look at the different responses and rank them based on their preference as to which one is better than the other. They use that to train a reward model so they can predict (basically using a different network) how much desirable would each response be.

(3) Once they have a reward model, they run PPO which is a form of policy gradient reinforcement learning optimizer to fine-tune this sampling policy such that the answers that ChatGPT generates are expected to score a high reward according to the reward model.

They have a whole alignment/fine-tuning stage with multiple steps, which takes the model from being a document completer to a question answerer. This is a whole separate stage, and a lot of this data is not available publically, as it's internal to OpenAI. It's much harder to replicate this stage than the pre-training stage. Anytime we don't just want a document completer (we want the model to perform tasks, we want it to be aligned in a specific way, we want it to predict sentiment, etc.), we have to complete further stages of fine-tuning which we didn't cover. That could be simple supervised fine-tuning or something more fancy like we've seen in ChatGPT (train a reward model, do rounds of PPO to align the model wrt. the reward model).

This is roughly what would give us a ChatGPT. nanoGPT just focuses on the pretraining stage.